In [2]:
from lib.main_func_p1 import timer, path
import pandas as pd
from importlib import reload
from collections import OrderedDict
import lib.main_func_p4
from lib.main_func_p4 import modelXGBoost_fit_scores
import os

#XGBoost library
import xgboost as xgb

In [3]:
#####################################
# proteina (uniprot_ID)
uniprot_id = 'P22303'
path_file = path(uniprot_id)
# Parametros
seed = 142854
fp_name = 'morgan2_c'

# Lista de resultados BayesSearchCV
results_list =  os.listdir(f'./grid_results/{uniprot_id}')
results_list

['P22303_20220617155153_BayesSearchCV_XGBoots_accuracy_rf0.xlsx',
 'P22303_20220620044412_BayesSearchCV_XGBoots_precision_rf0.xlsx',
 'P22303_20220621115528_BayesSearchCV_XGBoots_balanced_accuracy_rf0.xlsx',
 'P22303_20220621115847_BayesSearchCV_XGBoots_jaccard_rf0.xlsx',
 'P22303_20220621115850_BayesSearchCV_XGBoots_f1_weighted_rf0.xlsx']

In [6]:
# Load train and validation datasets
df_train = pd.read_pickle(f'{path_file}_dataset_train')
df_valid = pd.read_pickle(f'{path_file}_dataset_valid')

# Cargar archivo / eliminar columnas innecesarias
for name in results_list:
    excel_name = f'./grid_results/{uniprot_id}/{name}'
    metric_score = '_'.join(str.split(excel_name,'_')[5:-1])
    print(excel_name)
    print(f'model_{metric_score}')

    result_len = 5
    resample_factor = int(str.split(excel_name,'_')[-1][2:-5])
    resample_mode = 'under_sampling'

    df_ori = pd.read_excel(excel_name, sheet_name=0)
    df_grid_results = df_ori[['params', 'rank_test_score']]
    df_grid_results = df_grid_results.drop_duplicates()
    df_grid_results.sort_values(by=['rank_test_score'], ascending=True, inplace=True)
    df_grid_results.reset_index(drop=True, inplace=True)

    for i, params_dict in enumerate(df_grid_results.params.iloc[0:result_len]):
        eval_metric = ['error', 'auc']
        tick = timer()
        params_dict = dict(eval(params_dict))
        default_params_xgb = {'booster': 'gbtree', 'tree_method': 'gpu_hist',
                              'objective':'binary:logistic', 'grow_policy': 'depthwise',
                              'eval_metric': eval_metric, 'early_stopping_rounds':10}
        params_dict.update(default_params_xgb)


        xgb_clf = xgb.XGBClassifier(**params_dict)

        # Train model and evaluating scores (train / validation)
        xgb_clf, scores_train, scores_valid = modelXGBoost_fit_scores(xgb_clf, fp_name, df_train, df_valid,
                                                                      resample_factor=resample_factor,
                                                                      resample_mode=resample_mode)

        xgb_clf.save_model(f'./models_test/{uniprot_id}/model_grid_{metric_score}_{i}.ubj')

        print(f'{i+1}/{result_len}. AUC_score=(train={scores_train[0]},'
          f' valid={scores_valid[0]}). Time elapsed: {timer(tick)}')

./grid_results/P22303/P22303_20220617155153_BayesSearchCV_XGBoots_accuracy_rf0.xlsx
model_accuracy
1/5. AUC_score=(train=0.995, valid=0.963). Time elapsed: 3.4 seconds.
2/5. AUC_score=(train=0.991, valid=0.958). Time elapsed: 3.0 seconds.
3/5. AUC_score=(train=0.994, valid=0.964). Time elapsed: 3.4 seconds.
4/5. AUC_score=(train=0.994, valid=0.962). Time elapsed: 3.3 seconds.
5/5. AUC_score=(train=0.991, valid=0.958). Time elapsed: 2.7 seconds.
./grid_results/P22303/P22303_20220620044412_BayesSearchCV_XGBoots_precision_rf0.xlsx
model_precision
1/5. AUC_score=(train=0.993, valid=0.963). Time elapsed: 4.7 seconds.
2/5. AUC_score=(train=0.994, valid=0.961). Time elapsed: 4.9 seconds.
3/5. AUC_score=(train=0.993, valid=0.96). Time elapsed: 4.6 seconds.
4/5. AUC_score=(train=0.991, valid=0.956). Time elapsed: 4.0 seconds.
5/5. AUC_score=(train=0.992, valid=0.959). Time elapsed: 4.4 seconds.
./grid_results/P22303/P22303_20220621115528_BayesSearchCV_XGBoots_balanced_accuracy_rf0.xlsx
model_ba